# 🔄 Transfer Learning Sarrera

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Helburua

Notebook honetan **Transfer Learning** kontzeptua ikasiko dugu:
- Zer da Transfer Learning?
- Eredu aurre-entrenatuen erabilera
- Feature Extraction vs Fine-Tuning
- Eredu aurre-entrenatuen kargatze praktikoak

---

## 📚 Ikasiko duguna

1. ✅ Transfer Learning oinarriak
2. ✅ Eredu aurre-entrenatuen erabilera (VGG16, ResNet50, MobileNet)
3. ✅ Feature Extraction teknika
4. ✅ Fine-Tuning teknika
5. ✅ Kasu praktikoak

## 1️⃣ Zer da Transfer Learning?

**Transfer Learning** teknika bat da non aurrez entrenatutako eredu bat erabilzen den beste problema batetarako.

### 🔑 Abantailak

- ⚡ **Azkarragoa**: Ez dugu eredua zerotik entrenatu behar
- 📊 **Datu gutxiago**: Dataset txikiekin lan egin dezakegu
- 🎯 **Zehaztasun handiagoa**: Eredu aurre-entrenatua ImageNet bezalako dataset erraldoiekin entrenatuta dago
- 💰 **Kostu baxuagoa**: Konputazio-baliabide gutxiago behar ditugu

### 📝 Motak

1. **Feature Extraction**: Eredu aurre-entrenatua erabili ezaugarriak ateratzeko
2. **Fine-Tuning**: Eredu aurre-entrenatua doitu gure problemarako

---

### 🖼️ Feature Extraction vs Fine-Tuning

| Feature Extraction | Fine-Tuning |
|-------------------|-------------|
| Convolutional layers izoztuta | Convolutional layers entrenatzen dira |
| Dense layers soilik entrenatzen | Dense + Conv layers entrenatzen |
| Dataset txikiekin | Dataset handiekin |
| Azkarragoa | Motela |

## 2️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegi nagusiak
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.applications import VGG16, ResNet50, MobileNetV2
from keras import layers, Model
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

# Estiloa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Liburutegiak kargatuta!")
print(f"📦 TensorFlow bertsioa: {keras.__version__}")

## 3️⃣ Eredu Aurre-Entrenatuen Kargatze Adibideak

In [ ]:
# VGG16 eredua kargatu
print("🔄 VGG16 eredua kargatzen...")
vgg16_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

print(f"✅ VGG16 kargatuta!")
print(f"   Geruza kopurua: {len(vgg16_base.layers)}")
print(f"   Parametro kopurua: {vgg16_base.count_params():,}")

In [ ]:
# ResNet50 eredua kargatu
print("\n🔄 ResNet50 eredua kargatzen...")
resnet_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

print(f"✅ ResNet50 kargatuta!")
print(f"   Geruza kopurua: {len(resnet_base.layers)}")
print(f"   Parametro kopurua: {resnet_base.count_params():,}")

In [ ]:
# MobileNetV2 eredua kargatu (mugikorreko eredua)
print("\n🔄 MobileNetV2 eredua kargatzen...")
mobilenet_base = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

print(f"✅ MobileNetV2 kargatuta!")
print(f"   Geruza kopurua: {len(mobilenet_base.layers)}")
print(f"   Parametro kopurua: {mobilenet_base.count_params():,}")

## 4️⃣ Eredu Arkitekturak Konparatu

In [ ]:
# Ereduak konparatu
ereduak = {
    'VGG16': vgg16_base,
    'ResNet50': resnet_base,
    'MobileNetV2': mobilenet_base
}

print("📊 Ereduen konparaketa:\n")
print(f"{'Eredua':<15} {'Geruzak':<10} {'Parametroak':<15} {'Tamaina (MB)':<15}")
print("="*60)

for izena, eredua in ereduak.items():
    parametroak = eredua.count_params()
    geruzak = len(eredua.layers)
    # Tamaina kalkulatu (aproximatua)
    tamaina_mb = parametroak * 4 / (1024 * 1024)  # 4 bytes parametro bakoitzeko
    
    print(f"{izena:<15} {geruzak:<10} {parametroak:<15,} {tamaina_mb:<15.2f}")

## 5️⃣ Feature Extraction Adibidea

In [ ]:
# Feature Extraction eredua sortu VGG16rekin
def sortu_feature_extraction_eredua(base_model, num_classes=10):
    """
    Feature Extraction eredua sortu base_model-etik
    """
    # Base model-a izoztu
    base_model.trainable = False
    
    # Eredu berria sortu
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Eredua sortu
feature_extraction_model = sortu_feature_extraction_eredua(vgg16_base, num_classes=10)

# Laburpena
feature_extraction_model.summary()

print("\n✅ Feature Extraction eredua sortuta!")
print(f"   Entrenatzen diren parametroak: {sum([np.prod(p.shape) for p in feature_extraction_model.trainable_weights]):,}")

## 6️⃣ Fine-Tuning Adibidea

In [ ]:
# Fine-Tuning eredua sortu
def sortu_fine_tuning_eredua(base_model, num_classes=10, fine_tune_from=100):
    """
    Fine-Tuning eredua sortu base_model-etik
    """
    # Base model-a izoztu
    base_model.trainable = True
    
    # Azken geruzak soilik entrenatuko dira
    for layer in base_model.layers[:fine_tune_from]:
        layer.trainable = False
    
    # Eredu berria sortu
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Eredua sortu
fine_tuning_model = sortu_fine_tuning_eredua(
    VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    num_classes=10,
    fine_tune_from=15  # Azken 4 blokeak entrenatuko dira
)

# Laburpena
fine_tuning_model.summary()

print("\n✅ Fine-Tuning eredua sortuta!")
print(f"   Entrenatzen diren parametroak: {sum([np.prod(p.shape) for p in fine_tuning_model.trainable_weights]):,}")

## 7️⃣ Geruzak Bistaratu

In [ ]:
# VGG16 geruzak bistaratu
print("📊 VGG16 geruzak:")
print(f"{'#':<5} {'Izena':<30} {'Mota':<25} {'Trainable':<10}")
print("="*75)

for i, layer in enumerate(vgg16_base.layers):
    print(f"{i:<5} {layer.name:<30} {layer.__class__.__name__:<25} {layer.trainable}")

# Zenbat geruza entrenatuko dira?
trainable_count = sum([1 for layer in vgg16_base.layers if layer.trainable])
print(f"\n✅ Entrenatzen diren geruzak: {trainable_count}/{len(vgg16_base.layers)}")

## 8️⃣ Data Preprocessing Transfer Learning-erako

In [ ]:
# ImageNet pre-prozesatzea
from keras.applications.vgg16 import preprocess_input

# Adibide bat
img_array = np.random.randint(0, 255, (1, 224, 224, 3)).astype('float32')

print("🖼️ Irudia pre-prozesatu aurretik:")
print(f"   Shape: {img_array.shape}")
print(f"   Min: {img_array.min():.2f}, Max: {img_array.max():.2f}")
print(f"   Mean: {img_array.mean():.2f}")

# Pre-prozesatu
img_preprocessed = preprocess_input(img_array.copy())

print("\n🖼️ Irudia pre-prozesatu ondoren:")
print(f"   Shape: {img_preprocessed.shape}")
print(f"   Min: {img_preprocessed.min():.2f}, Max: {img_preprocessed.max():.2f}")
print(f"   Mean: {img_preprocessed.mean():.2f}")

print("\n✅ Pre-prozesatzea aplikatuta!")

## 9️⃣ Aurreikuspenak ImageNet Klaseekin

## 📝 Ondorioak

### 🎯 Ikasitako Kontzeptuak

- **Transfer Learning**: Eredu aurre-entrenatuen erabilera
- **Feature Extraction**: Base model izoztuta, dense layers entrenatuz
- **Fine-Tuning**: Azken convolutional layers entrenatuz
- **Eredu desberdinak**: VGG16, ResNet50, MobileNetV2

### 📊 Ereduen Konparaketa

| Eredua | Geruzak | Parametroak | Erabilera |
|--------|---------|-------------|-----------|
| VGG16 | 19 | 138M | Zehaztasun handia |
| ResNet50 | 50 | 25M | Sakontasun handia |
| MobileNetV2 | 53 | 3.5M | Mugikorreko aplikazioak |

### 🚀 Hurrengo Pausoak

- **Kasu praktikoa**: Cats vs Dogs klasifikazioa
- **Data Augmentation**: Overfitting-a ekiditeko
- **Fine-Tuning**: Azken geruzak optimizatu
- **Eredu konparaketa**: Eredu desberdinak frogatu

---

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Lizentzia**: MIT